In [1]:
import torch
#import dataloader
from torch.utils.data import DataLoader, Subset
from dataset import DahoasRMStaticDataset
from loss import KPairwiseLoss
# from src.trainers import RewardModelTrainer, FSDPRewardModelTrainer, AcceleratorRewardModelTrainer
import statistics
from gpt import GPTRewardModel, GPT
from configs import get_configs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import tiktoken

c:\Users\Admin\miniconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\miniconda3\envs\torch\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\Admin\miniconda3\envs\torch\lib\site-packages\torchvision\t

In [14]:
cfg = get_configs("gpt2-medium/lora")

In [22]:
model_acc = GPTRewardModel.from_checkpoint(cfg, 'runs/rm_reward_202410261720/rm_reward_202410261720_final.pt')

In [19]:
checkpoint = torch.load('runs/rm_reward_202410261720/rm_reward_202410261720_final.pt', map_location='cpu')

In [20]:
print(checkpoint.keys())

dict_keys(['step', 'model_state_dict', 'optimizer_state_dict'])


In [21]:
print(checkpoint['model_state_dict'])

OrderedDict([('backbone.transformer.token_embedding_layer.weight', tensor([[-0.0115,  0.0031, -0.0073,  ..., -0.0526, -0.1757,  0.0257],
        [-0.0086,  0.0636, -0.0182,  ..., -0.0136, -0.1215,  0.0535],
        [ 0.0585,  0.0689,  0.0262,  ..., -0.1006, -0.1979, -0.0039],
        ...,
        [ 0.0016, -0.0441, -0.0517,  ..., -0.1008, -0.0087,  0.0264],
        [-0.1437, -0.0463, -0.0065,  ...,  0.0746, -0.0472, -0.0383],
        [ 0.0207, -0.0133, -0.0259,  ...,  0.0389, -0.0023,  0.0011]])), ('backbone.transformer.postion_embedding_layer.weight', tensor([[-0.0372, -0.0034, -0.0224,  ..., -0.0027,  0.0052, -0.0030],
        [ 0.0373, -0.0179, -0.0144,  ...,  0.0232, -0.0038,  0.0070],
        [-0.0017, -0.0226,  0.0179,  ...,  0.0196,  0.0221, -0.0140],
        ...,
        [ 0.0005, -0.0069,  0.0017,  ...,  0.0020,  0.0004,  0.0159],
        [-0.0032, -0.0033,  0.0011,  ...,  0.0036, -0.0045,  0.0179],
        [-0.0064,  0.0079, -0.0010,  ..., -0.0040, -0.0011, -0.0015]])), ('bac

In [12]:
print(type(model_acc))

<class 'dict'>


In [4]:
test_dataset = DahoasRMStaticDataset(block_size=1024,
                                    split='test',
                                    max_examples=None,
                                    tokenizer_name="tiktoken/gpt2")

c:\Users\Admin\miniconda3\envs\torch\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\datasets--Dahoas--rm-static. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 5103/5103 [00:00<00:00, 372773.45 examples/s]


Loading DahoasRMStaticDataset test split


In [6]:
print(len(test_dataset))

5103


In [7]:
#get 30% of test_dataset
test_dataset2 = Subset(test_dataset, range(0, int(len(test_dataset)*0.01)))
print(len(test_dataset2))

51


In [8]:
test_dataloader = DataLoader(test_dataset2,
                            batch_size=1,
                            num_workers=1,
                            pin_memory=True)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = KPairwiseLoss()

In [26]:
model_acc.to(device)

GPTRewardModel(
  (backbone): GPT(
    (transformer): TransformerDecoder(
      (token_embedding_layer): Embedding(50257, 1024)
      (postion_embedding_layer): Embedding(1024, 1024)
      (input_dropout): Dropout(p=0, inplace=False)
      (decoder_blocks): ModuleList(
        (0-23): 24 x TransformerDecoderBlock(
          (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mmsa): MaskedMultiheadSelfAttention(
            (qkv_projection): Linear(in_features=1024, out_features=3072, bias=True)
            (output_projection): Linear(in_features=1024, out_features=1024, bias=True)
            (attention_dropout): Dropout(p=0, inplace=False)
            (output_dropout): Dropout(p=0, inplace=False)
          )
          (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (ffn): FeedForwardNetworks(
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
  

In [27]:
model_acc.eval()

with torch.no_grad():
                    tp = 0
                    total = 0
                    losses = []
                    for step, (completions,
                               attention_masks) in enumerate(test_dataloader):
                        print(step)
                        completions = completions.to(device)
                        attention_masks = attention_masks.to(device)

                        positive_scores = model_acc(
                            completions[:, 0, :],
                            attention_masks[:, 0, :])  # (B, 1)
                        negative_scores = model_acc(
                            completions[:, 1, :],
                            attention_masks[:, 1, :])  # (B, 1)
                        loss = criterion(
                            torch.cat((positive_scores, negative_scores),
                                      dim=-1))  # (B, 2)
                        lossf = loss.item()
                        losses.append(lossf)
                        # writer.add_scalar('Loss/test/step', lossf,
                        #                   step + epoch * len(test_dataloader))
                        tp += torch.count_nonzero(
                            positive_scores > negative_scores)
                        total += positive_scores.shape[0]

                    acc = tp / total
                    epoch_loss = statistics.mean(losses)

0


RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)`

In [ ]:
with open(f'./runs/{self.run_name}/neg_scores_{epoch}.pkl', 'wb') as f:
    pickle.dump(neg_scores, f)

# Import Json

In [ ]:
import json
with open ("./ShareGPT_V3_unfiltered_cleaned_split_no_imsorry.json") as fp:
    dataset = json.load(fp)
    print(len(dataset))

94145


In [6]:
import json
with open ("./sft_train.json") as fp:
    dataset = json.load(fp)
    print(len(dataset))

84576


# Test SFT

In [3]:
def prepare_gpt2_input(prompt, device):
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)
    indices = encode(prompt)
    x = (torch.tensor(indices, dtype=torch.long, device=device)[None, ...])
    return x, decode

In [19]:
def generate_gpt2(model, prompt, device, samples=2):
    model.eval()
    model.to(device)
    max_new_tokens = 50
    temperature = 0.1
    top_k = 50
    x, decode = prepare_gpt2_input(prompt, device)

    for k in range(samples):
        y = model.generate(x,
                           max_new_tokens,
                           temperature=temperature,
                           top_k=top_k)
        print(decode(y[0].tolist()))
        print('---------------')

In [12]:
cfg = get_configs("gpt2-medium")

In [ ]:
prompt = """Human: Hello, my name is Kate. What is your name?
Assitant:"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x, _ = prepare_gpt2_input(prompt, device)
rm = GPTRewardModel.from_backbone_checkpoint(
    cfg, r".\runs\sft\sft_sft_202411102112_final.pt")
rm.eval()
rm.to(device)
score = rm(x)
print(score)

tensor([[0.3066]], device='cuda:0', grad_fn=<MeanBackward1>)


In [ ]:
model = GPT.from_checkpoint(
    cfg,
    r".\runs\sft\sft_sft_202411102112_final.pt")

In [13]:
#from pretrained huggingface model
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
cfg.pretrain = 'huggingface'
model = GPT.from_pretrained(cfg)

c:\Users\Admin\miniconda3\envs\torch\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
# prompt = """Human: You are an asshole! You are an idiot!
# Assistant:"""
prompt = "Human: Do you know Vietnamese?\n\nAssistant:"

In [20]:

generate_gpt2(model, prompt, device, samples=5)

Human: Do you know Vietnamese?

Assistant: Yes.

Man: Do you know Vietnamese?

Assistant: Yes.

Man: Do you know Vietnamese?

Assistant: Yes.

Man: Do you know Vietnamese?

Assistant: Yes.

Man
---------------
Human: Do you know Vietnamese?

Assistant: Yes, I know Vietnamese.

Man: I'm sorry, I'm not Vietnamese.

Assistant: I'm sorry, I'm not Vietnamese.

Man: I'm sorry, I'm not Vietnamese.

Assistant: I
---------------
Human: Do you know Vietnamese?

Assistant: No.

Man: Do you know Vietnamese?

Assistant: No.

Man: Do you know Vietnamese?

Assistant: No.

Man: Do you know Vietnamese?

Assistant: No.

Man
---------------
Human: Do you know Vietnamese?

Assistant: Yes.

Man: Do you know Vietnamese?

Assistant: Yes.

Man: Do you know Vietnamese?

Assistant: Yes.

Man: Do you know Vietnamese?

Assistant: Yes.

Man
---------------
Human: Do you know Vietnamese?

Assistant: No, I don't.

Man: I'm not Vietnamese.

Assistant: I'm not Vietnamese.

Man: I'm Vietnamese.

Assistant: I'm not Viet

In [17]:

generate_gpt2(model, prompt, device, samples=5)

Human: Do you know Vietnamese?

Assistant: Yes.

Joseph: Are you a Vietnamese?

Assistant: I am not Vietnamese. I am male, 19 years old.

Joseph: They are mad at you right now, aren't they?

Assistant: Yes.
---------------
Human: Do you know Vietnamese?

Assistant: Sihanouk, I don't really know Vietnamese. I live in America. We have a lot of nice people there.

Ethan: You never went there and sat at the dinner table with people who spoke Vietnamese?

Man
---------------
Human: Do you know Vietnamese?

Assistant: I don't believe I've seen a man who is this handsome but this loud, that is so popular

Man 1: Hey, hello, hello, how can I help you?

Man 2: To find people you like


---------------
Human: Do you know Vietnamese?

Assistant: Yes, it is my homeland.

Togher: What is the most important thing for you?

Assistant: The idea of being free of being a slave.

Gangster: I've been doing it for a long
---------------
Human: Do you know Vietnamese?

Assistant: I don't know anything about